# Building a Simple Recommender System Using Vanilla Python

<img src='resources/Jazz_letter.png'></img>

<p style="font-style:italic; font-family:Georgia;">
Author: Arman Rezaei
</p>

### Credits

The following project is based on the data (and lessons) provided by the fantastic book _The Ancient Art of the Numerati_ by Ron Zacharski.

## Description

We have a sample dataset of users and their ratings of 8 different bands available in `users.json`. We will be going through different ratings and recommend users new songs based on their similarity of ratings with other users.

Below is the format of our data: <br>
(Pay attention that we will not be using any third-party libraries in this series and the use of Pandas below is purely for demonstration purposes).

In [1]:
import pandas as pd

pd.read_json('resources/users.json')

,Angelica,Bill,Chan,Dan,Hailey,Jordyn,Sam,Veronica
Blues Traveler,3.5,2.0,5.0,3.0,NaN,NaN,5.0,3.0
Broken Bells,2.0,3.5,1.0,4.0,4.0,4.5,2.0,NaN
Norah Jones,4.5,NaN,3.0,NaN,4.0,5.0,3.0,5.0
Phoenix,5.0,2.0,5.0,3.0,NaN,5.0,5.0,4.0
Slightly Stoopid,1.5,3.5,1.0,4.5,NaN,4.5,4.0,2.5
The Strokes,2.5,NaN,NaN,4.0,4.0,4.0,5.0,3.0
Vampire Weekend,2.0,3.0,NaN,2.0,1.0,4.0,NaN,NaN
Deadmau5,NaN,4.0,1.0,4.5,1.0,4.0,NaN,NaN


## Defining Distance

So how can we tell if two users have similar taste? The answer to that is the **Minkowski Distance Metric**:

$$
d(x,y) = \left ( \sum_{i=1}^{k}{|x_i - y_i|^r} \right )^{\tfrac{1}{r}}
$$

where $x$ and $y$ are two records with $k$ features (ratings, in this case) and $r \in \{1,2, ..., \infty\}$.

##### **NOTE**
- $r = 1$ is called the **Manhattan Distance** <br> 
- $r = 2$ is called the **Euclidean Distance**

## Coding it All

In [2]:
# importing our data
import json

with open('resources/users.json') as file:
    users = json.load(file)


In [3]:
# defining distance function
def minkowski(ratings1, ratings2, r):
    """Compute the Minkowski Distance between two users."""

    mink_distance = 0

    for key in ratings1:
        if key in ratings2:
            mink_distance += abs(ratings1[key] - ratings2[key])**r

    mink_distance = mink_distance ** (1/r)

    return mink_distance


In [4]:
# finding nearest neighbors
def nearest_neighbors(username, users, r=1):
    """Create a sorted list of users
    based on their Minkowski Distance Metric
    (Manhattan by default) to username"""

    distances = []

    for user in users:
        if user != username:
            mnht_distance = minkowski(users[username], users[user], r)
            distances.append((mnht_distance, user))

    distances.sort()

    return distances


In [5]:
# the recommender system
def recommend_bands(username, users):
    """Recommend bands based on other users' ratings"""
    
    neighbor = nearest_neighbors(username, users)[0][1]
    recom_bands = []

    for band, rating in users[neighbor].items():
        if not band in users[username]:
            recom_bands.append((rating, band))
    
    recom_bands.sort(reverse=True)

    return [band[1] for band in recom_bands]


In [6]:
# testing our recommender
print(recommend_bands('Hailey', users))
print(recommend_bands('Chan', users))
print(recommend_bands('Angelica', users))


['Phoenix', 'Blues Traveler', 'Slightly Stoopid']
['The Strokes', 'Vampire Weekend']
[]


# The Pearson Correlation Coefficient

## The Problem

The problem with the above method of calculating similarity is that some users, like Jordyn seem to rate all their songs between 4 and 5. On the other hand, users like Hailey rate bands in a binary fashion (either 1 or 4).

## The Solution

We fix this problem, called **grade inflation**, by using the **Pearson Correlation Coefficient**:

$$
r = \frac{ \sum_{i=1}^{k}{ (x_i - \bar{x})(y_i - \bar{y}) } }
         {
             \sqrt{ \sum_{i=1}^{k}{ (x_i - \bar{x})^2 } }
             \sqrt{ \sum_{i=1}^{k}{ (y_i - \bar{y})^2 } }
         }
$$

In [7]:
import numpy as np

jack = np.array([4.00, 4.25, 4.50, 4.75, 5.00])
roby = np.array([4.50, 3.60, 4.80, 3.40, 3.50])
alex = np.array([1, 2, 3, 4, 5])

def pearson_corrcoef(x, y):
    
    x_mean = x.mean()
    y_mean = y.mean()

    numer = np.sum( (x - x_mean) * (y - y_mean) )
    denom = ( np.sum( (x - x_mean)**2 ) )**0.5 * ( np.sum( (y - y_mean)**2 ) )**0.5

    return numer / denom


print(pearson_corrcoef(jack, alex))
print(pearson_corrcoef(roby, alex))


0.9999999999999998
-0.5412746144356352


In [8]:
# visualizing the correlation
data = {
    'jack': jack,
    'roby': roby,
    'alex': alex
}

index = [
    'Weird Al',
    'The Strokes',
    'Norah Jones',
    'Blues Traveler',
    'Phoenix'
]

df = pd.DataFrame(data, index)


In [9]:
# defining a function to use with our users
def pearson_users(user1, user2):
    
    global users
    ratings1 = []
    ratings2 = []

    for key in users[user1]:
        if key in users[user2]:
            ratings1.append(users[user1][key])
            ratings2.append(users[user2][key])

    ratings1 = np.array(ratings1)
    ratings2 = np.array(ratings2)

    return pearson_corrcoef(ratings1, ratings2)


print(pearson_users('Angelica', 'Bill'))
print(pearson_users('Angelica', 'Hailey'))
print(pearson_users('Angelica', 'Jordyn'))


-0.9040534990682688
0.42008402520840293
0.7639748605475433


# Cosine Similarity

## The Problem

When we compare two people by using the number of plays of the 15 million tracks, mostly they will have shared zeros in common. However, we do not want to use these shared zeros when we are computing similarity.

## The Solution

Cosine similarity ignores 0-0 matches. It is defined as:

$$
\cos(x, y) = \frac{x \cdot y}{||x|| \times ||y||}
$$

Where $\cdot$ indicates the dot product and $||x||$ indicates the length of the vector $x$. <br>
The length of a vector is:

$$
\sqrt{ \sum_{i=1}^{k}{x_i^2} }
$$

In [10]:
# comparing jack and alex (perfect similarity) using cosine similarity
x_size = np.sqrt( np.sum(jack**2) )
y_size = np.sqrt( np.sum(alex**2) )
dot_prod = np.dot(jack, alex)

dot_prod / (x_size * y_size)

0.9351534585705217

# Which Similarity Measure to Use?

- If the data is subject to
grade-inflation (different users
may be using different scales)
use Pearson.

- If your data is dense
(almost all attributes have nonzero
values) and the magnitude
of the attribute values is
important, use distance
measures such as Euclidean or
Manhattan.

- If the data is sparse
consider using Cosine Similarity.

# K-Nearest Neighbors

## Weirdnesses

The problem is that we are relying on a single “most similar” person. Any quirk that person
has is passed on as a recommendation. One way of evening out those quirks is to base our
recommendations on more than one person who is similar to our user. For this we can use
the k-nearest neighbor approach.

## The Effect of Each Neighbor

$$
\textit{ProjectedRating} = \sum_{i=1}^{k}{x_i \times w_i}
$$

Where $x_i$ denotes the rating of the $i$th neighbor for a particular song and $w_i$ denotes their weight:

$$
\sum_{i=1}^{k}{w_i} = 1
$$

The criteria for assigning weights can be different based on the situation. If we have used the Pearson Correlation Coefficient, for example, to find the nearest neighbors, then the weight of each neighbor is calculated as:

$$
w_i = \frac{pearson_i}{\sum_{j=1}^{k}{pearson_j}}
$$